# Simple greedy policy

This simple greedy policy ($\Pi_{SG}$) tries to play game with the best action possible in a given moment with minimal random steps. We are going to look at following metrics:
 - $\mathbb{E}(X)=\frac{76}{15} \approx 5.06666$ - expected value when all cards are unknown
 - $\mathbb{E}(X_t) \in [-2, 12]$ - expected value of all _unknown_ cards at time $t$
 - $V(\cdot) \in [-2, 12]$ - value of a card
 - $V(discard\ pile)$ - value of top card in discard pile
 - $V(drawn\ card)$ - value of a drawn card
 - $\max_{x \in \mathcal{B}}V(x)$ - highest card value on the board (board is 12 cards in front of each player)
   - if card is not turned i.e. unknown value $V(x) = \mathbb{E}(X_t)$ 
 - $\mathcal{R_i} \in [0,12]$ - number of remaining (_unknown_) cards on $i$-th player's board

## Policy flow
1. draw a card from discard pile ($V(discard\ pile)$) or deck $\mathbb{E}(X_t)$, which one is lower
2. replace card with the highest difference between drawn card and any card on the board, unknown cards has value $\mathbb{E}(X_t)$, i.e. $$\max_{x\in\mathcal{B}}\big[V(x) - V(drawn\ card)\big]$$ If more cards have the same value, pick one of them at random.
3. repeat till all cards are turned or someone else ended the game